# Delineating Chromophores With VMD

Running MorphCT requires the specification of chromophores within the morphology. That is, where do photo-excited electrons (or holes) delocalize and present themselves as potential hoppers onto neighboring chromophores. Every molecule is different and this picking is a bit of trial and error at the moment. This notebooks seeks to dramatically speed up that trial and error time. We take snapshots from a 10 molecule MD simulation of ITIC and we show how you can use VMD to specifiy chromophores quickly. Literature suggest that, for ITIC, that charges delocalize primarily along the backbone. It is for this reason that we demonstrate how we can use this workflow to specifiy the backbone as the chromophore. 

In [1]:
from copy import deepcopy
import os
import re
import gsd.hoomd
import mbuild as mb
import numpy as np
from morphct import execute_qcc as eqcc
from morphct import chromophores
from morphct import kmc_analyze
from morphct.chromophores import conversion_dict
from morphct.chromophores import amber_dict
from morphct.mobility_kmc import snap_molecule_indices

def visualize_qcc_input(qcc_input):
    """
    Visualize a quantum chemical input string (for pyscf) using mbuild.
    
    Parameters
    ----------
    qcc_input : str
        Input string to visualize
    """
    comp = mb.Compound()
    for line in qcc_input.split(";")[:-1]:
        atom, x, y, z = line.split()
        xyz = np.array([x,y,z], dtype=float)
        # Angstrom -> nm
        xyz /= 10
        comp.add(mb.Particle(name=atom,pos=xyz))
    comp.visualize().show()
    
def from_snapshot(snapshot, scale=1.0):
    """
    Convert a hoomd.data.Snapshot or a gsd.hoomd.Snapshot to an
    mbuild Compound.
    
    Parameters
    ----------
    snapshot : hoomd.data.SnapshotParticleData or gsd.hoomd.Snapshot
        Snapshot from which to build the mbuild Compound.
    scale : float, optional, default 1.0
        Value by which to scale the length values
        
    Returns
    -------
    comp : mb.Compound
    """
    comp = mb.Compound()
    bond_array = snapshot.bonds.group
    n_atoms = snapshot.particles.N

    # There will be a better way to do this once box overhaul merged
    try:
        # gsd
        box = snapshot.configuration.box
        comp.box = mb.box.Box(lengths=box[:3] * scale)
    except AttributeError:
        # hoomd
        box = snapshot.box
        comp.box = mb.box.Box(lengths=np.array([box.Lx,box.Ly,box.Lz]) * scale)

    # to_hoomdsnapshot shifts the coords, this will keep consistent
    shift = np.array(comp.box.lengths)/2
    # Add particles
    for i in range(n_atoms):
        name = snapshot.particles.types[snapshot.particles.typeid[i]]
        xyz = snapshot.particles.position[i] * scale + shift
        charge = snapshot.particles.charge[i]

        atom = mb.Particle(name=name, pos=xyz, charge=charge)
        comp.add(atom, label=str(i))

    # Add bonds
    particle_dict = {idx: p for idx, p in enumerate(comp.particles())}
    for i in range(bond_array.shape[0]):
        atom1 = int(bond_array[i][0])
        atom2 = int(bond_array[i][1])
        comp.add_bond([particle_dict[atom1], particle_dict[atom2]])
    return comp

def vmd_index_slicer(filepath):
    with open(filepath, "r") as f:
        index_ids = f.readlines()
    numbers = []
    
    for line in index_ids:
        thing = re.findall("\/([0-9]+)(?=[^\/]*$)", line)
        try:
            numbers.append(int(thing[0]))
        except:
            pass
        
    indices=[]
    
    for i in numbers:
        if i not in indices:
            indices.append(i)
            
    return indices

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

/Users/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/h5py/__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters


To begin we can take a look at the trajectory.

In [2]:
gsd_file = "itic-trajectory.gsd"
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    start_snap = f[0]
    end_snap = f[-1]

    
box = start_snap.configuration.box[:3]
ref_distance = 3.563594872561358
unwrapped_positions = start_snap.particles.position + start_snap.particles.image * box
start_snap.particles.position *= ref_distance
start_snap.configuration.box[:3] *= ref_distance
end_snap.particles.position *= ref_distance
end_snap.configuration.box[:3] *= ref_distance
unwrap_snap = deepcopy(start_snap)
unwrap_snap.particles.position = unwrapped_positions
unwrap_snap.particles.types = [amber_dict[i].symbol for i in start_snap.particles.types]
comp = from_snapshot(unwrap_snap, scale=0.1*ref_distance)
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

At this point, navigate to the examples/ directory in the command line and open itic-trajectory.gsd
with VMD by typing the following command:  `vmd itic-trajectory.gsd`
1. If VMD isn't installed see this page for instructions. https://github.com/cmelab/getting-started/blob/master/wiki/pages/How_to_visualize.md
2. With the trajectory open, type the following in the command line: `logfile chomophores.txt`.
This tells VMD to save the information from vmd to a file called chromophores.txt
3. navigate to the vmd display, scroll to the beginning of the trajectory for viewing pleasure, and hit the 2 key on your keyboard. then click all the atoms that should belong to your chromophore.
4. Having done that we can set the chromo_ids below to be the indices for the atoms that clicked on and view them in pink with the cell below. Here we have chosen the backbone of ITIC to be a chromophore because this has been shown to be the where charges delocalize. The side chains affect charge mobility as well but largely through their effects on packing and morphology. 

NOTE: when clicking atoms for use with the following function, it doesn't hurt to click an atom multiple times to insure its input into the file. The slicer function ignores duplicates. 

In [3]:
chromo_ids = vmd_index_slicer("chromophores.txt")
for i,p in enumerate(comp.particles()):
    if i in chromo_ids:
        p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

If, upon inspection, you discover that you have left out an atom that you want to include in your chromophore, head back to vmd and click it. If you still have VMD running, it should keep ammending the chromophores.txt file. Now we can use morphct to probe
the chosen chromopores HOMO/LUMO levels pre and post 
equilibration. As seen in the following two cells. 

In [4]:
qcc_input = eqcc.write_qcc_inp(start_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("Pre equillibrated HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


Pre equillibrated HOMO-1, HOMO, LUMO, LUMO+1
[-8.30593093 -7.33816216 -0.64400237 -0.27038863]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
qcc_input = eqcc.write_qcc_inp(end_snap, chromo_ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("post equillibration HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


post equillibration HOMO-1, HOMO, LUMO, LUMO+1
[-7.96082754 -6.83943147 -1.45624648 -0.78566962]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

What if you are not sure what to specify as a chromophore, or you dont want to bother indexing? Below we will take an entire molecule to 
be a chromophore using the cluster analysis provided by morphct. The following cell looks weird but it just figures out how many atoms are in each molecule (k), and takes the first "k" ids to be the chromophore, i.e. the entire first molecule.

In [6]:
gsd_mol_index = snap_molecule_indices(start_snap)
k = np.count_nonzero(gsd_mol_index==1)
ids = np.arange(start_snap.particles.N)[0:k]

With the new ids specified we can procede exactly as before. We show the HOMO/LUMO pre and post 
equilibration.

In [7]:
qcc_input = eqcc.write_qcc_inp(start_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("pre equillibration HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)


pre equillibration HOMO-1, HOMO, LUMO, LUMO+1
[-7.7267407  -7.22392908 -0.8465483  -0.66697131]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
qcc_input = eqcc.write_qcc_inp(end_snap, ids, amber_dict)
homolumo = eqcc.get_homolumo(qcc_input)
print("post equillibration HOMO-1, HOMO, LUMO, LUMO+1")
print(homolumo)
visualize_qcc_input(qcc_input)

post equillibration HOMO-1, HOMO, LUMO, LUMO+1
[-6.92726715 -6.26889544 -1.90024535 -1.54897818]


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

One final bit of useful logic is displayed below. Let's say you found the chromophore ids for a random molecule in a morpholgy but want to extraploate that accross the morphology. Using the same logic above for finding the number of atoms per molecule, We take the indices from the vmd_slicer and take that modulo that number. This gives the corresponding indices for the first molecule. 

In [9]:
indices = vmd_index_slicer("chromophores.txt")
gsd_mol_index = snap_molecule_indices(start_snap)
k = np.count_nonzero(gsd_mol_index==1)
for i in range(len(indices)):
    indices[i] = indices[i]%k
chromo_ids = np.array(indices)

 From there we can add "k" back in to get the corresponing atoms in the 2nd molecule , then the 3rd etcetera. This gives a master list of chromophere ids that we are interested in. Again we can visualize them. 

In [10]:
master_list = []
sublist = chromo_ids
for i in range(len(np.unique(gsd_mol_index))):         
    master_list.append(sublist)
    sublist = [x + k for x in sublist]
for x in range(len(master_list)):
    for i,p in enumerate(comp.particles()):
        if i in master_list[x]:
            p.name = "Kr"
comp.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Now you can pick novel chromophores upon which you can do analysis. In workflow-p3ht.ipynb there is an example of how to do this with smiles strings and also how to use this master_list to do complete mobility analysis of your morphology. But for big, or jumbled up, or brand new molecules this can be a time sink. (Or if you just prefer the clicky clicky!)